## The Python Data Model

The Python data model formalizes the interfaces of the building blocks of the language. The Python interpreter invokes special methods to perform basic object operations. The special method names are always spelled with leading and trailing double underscores, i.e. `__getitem__`.

The special method names allow your objects to interact with basic language constructs such as:
-  iteration
    -  **for**
    -  **while**
-  collections: This module implements specialized container datatypes providing alternatives to Python's general purpose built-in containers, dict, list, set, and tuple.
    -  **namedtuple()** factory function for creating tuple subclasses with named fields    
    -  **deque** list-like container with fast appends and pops on either end    
    -  **Counter** dict subclass for counting hashable objects 
    -  **OrderedDict** dict subclass that remembers the order entries were added   
    -  **defaultdict** dict subclass that calls a factory function to supply missing values
-  attribute access
    -  `object.attribute`
    -  `getattr(obj, 'attribute')`
-  operator overloading
    -  operator overloading is done by redefining certain special methods in any class eg"
        ```class MyClass(object):
         def __add__(self, x):
             return '%s plus %s' % (self, x)```
-  function and method invocation. Methods are associated with object instances or classes; functions aren't. 
    -  method invocation
        -  instance = Foo()
        -  instance.foo(arg1,arg2)
    -  function invocation:
        -  Bar.foo(1,2)
-  object creation and destruction;
    -  x = MyClass()
    -  object destruction via the __del__ method
-  string representation and formatting
    - name = "John"
    - print("Hello, %s!" % name)
-  managed contexts (i.e. with blocks)
    -  automatically manage resources encapsulated within context manager types, or more generally performs startup and cleanup actions around a block of code. eg:
    ```with open('what_are_context_managers.txt', 'r') as infile:
    for line in infile:
        print('> {}'.format(line))```



Nice [presentation of the Python data model](https://delapuente.github.io/presentations/python-datamodel/index.html#/4/1)

In [67]:
#Using collections here
import collections
from random import choice

#named tuple in use
Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA') 
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
                       
    def __len__(self):
        return len(self._cards)
                       
    def __getitem__(self, position): 
        return self._cards[position]

In [68]:
deck = FrenchDeck()
print(len(deck))
print(deck[0])
print(deck[-1])
print(choice(deck))

52
Card(rank='2', suit='spades')
Card(rank='A', suit='hearts')
Card(rank='J', suit='hearts')


By implementing the special methods `__len__` and `__getitem__` our FrenchDeck behaves like a standard Python sequence (i.e. str, unicode, list, tuple, buffer, xrange), allowing it to benefit from core language features — like iteration and slicing—and from the standard library

Thress advantages of using special methods to leverage the Python Data Model:
1. Standard operations have standard names, across objects
1. Once standard operations have been implemented, rest of the the library is available
1. Because `__getitem__` delegates to the [] operator of self._cards, our deck automatically supports slicing. 

In [69]:
print(deck[:3])
print(deck[12::13])
print(Card('Q', 'hearts') in deck)
print(Card('7', 'beasts') in deck)

[Card(rank='2', suit='spades'), Card(rank='3', suit='spades'), Card(rank='4', suit='spades')]
[Card(rank='A', suit='spades'), Card(rank='A', suit='diamonds'), Card(rank='A', suit='clubs'), Card(rank='A', suit='hearts')]
True
False


-  Special methods are meant to be called by the Python interpreter, and not by you.
-  The only special method that is frequently called by user code directly is __init__, to invoke the initializer of the superclass in your own __init__ implementation.
-  If you need to invoke a special method, it is usually better to call the related built-in function, such as len, iter, str etc.

## Emulating numeric types

In [4]:
from math import hypot 

class Vector:
    def __init__(self, x=0, y=0): 
        self.x = x
        self.y = y
    def __repr__(self):
        #note use of string representation, to get raw value, not str as %s
        return 'Vector(%r, %r)' % (self.x, self.y) 
    def __abs__(self):
        return hypot(self.x, self.y) 
    def __bool__(self):
        #Convert the magnitude to a boolean using bool(abs(self)) because __bool__ is expected to return a boolean.
        return bool(abs(self))
    def __add__(self, other):
        '''
        >>> v1 = Vector(2, 4) 
        >>> v2 = Vector(2, 1) 
        >>> v1+v2 
        Vector(4, 5)
        '''
        x = self.x + other.x 
        y = self.y + other.y 
        return Vector(x, y)
    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

import doctest
print(doctest.testmod())
v1 = Vector(2, 4)
v2 = Vector(2, 1) 
print(v1+v2)
v = Vector(3, 4)
print(abs(v))
print(v*3)
print(abs(v * 3))

TestResults(failed=0, attempted=3)
Vector(4, 5)
5.0
Vector(9, 12)
15.0


Note that although we implemented four special methods (apart from __init__), none of them is directly called

## String representation

The __repr__ special method is called by the repr built-in to get string representation of the object for inspection. Note that in our __repr__ implementation we used %r to obtain the standard repre‐ sentation of the attributes to be displayed. This is good practice, as it shows the crucial difference between Vector(1, 2) and Vector('1', '2') — the latter would not work in the context of this example, because the constructors arguments must be numbers, not str.

## Arithmetic Operators

There are 3 kinds of operators calling-notations: prefix (+ 3 5), infix (3 + 5), and postfix (3 5 +). The __add__ and __mul__ methods create and return a new instance of Vector, and do not modify either operand — self or other are merely read. This is the expected behavior of infix operators: to create new objects and not touch their operands.

## Boolean value of a custom type

To determine whether a value x is truthy or falsy, Python applies bool(x), which always returns True or False.

## Why len is not a method

len is not called as a method because it gets special treatment as part of the Python Data Model, just like abs. But thanks to the special method __len__ you can also make len work with your own custom objects. This is fair compromise between the need for efficient built-in objects and the consistency of the language. Also from the Zen of Python: “Special cases aren’t special enough to break the rules.”

# Summary

By implementing special methods, custom objects can behave like the built-in types, enabling Pythonic 'expressive coding'. Emulating sequences, as shown with the FrenchDeck example, is one of the most widely used applications of the special methods.

A basic requirement for a Python object is to provide usable string representations of itself, one used for debugging and logging, another for presentation to end users. That is why the special methods __repr__ and __str__ exist in the Data Model.